In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F
import sys
sys.path.append('../scripts')  # Assuming '../scripts' is the relative path to the module directory
from data_loader import load_data


# Set the device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load in data from data_loader.py
data_dir = "../snakes"
resize_option = False
loaders = load_data(data_dir, customized_size=resize_option)

In [ ]:
print('Number of training samples: {}'.format(len(loaders['train'].sampler)))
print('Number of validation samples: {}'.format(len(loaders['valid'].sampler)))
print('Number of test samples: {}'.format(len(loaders['test'].sampler)))

In [ ]:
# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(24, 8, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(8 * 64 * 64, num_classes)  # Adjusted based on the new resize dimensions

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16 * 64 * 64)
        x = self.fc1(x)
        return x

In [ ]:
# Initialize the model
model = SimpleCNN(num_classes=38).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

In [32]:
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in loaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print the average loss after each epoch
    #print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(loaders['train'])}')

# Validate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in loaders['test']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy * 100:.2f}%')